In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier

from datetime import datetime

In [2]:
# Load the data sets
data_dir = './data/'
df_Household = pd.read_csv(data_dir + 'Household.csv')
df_Trip = pd.read_csv(data_dir + 'Trip.csv')
df_Person = pd.read_csv(data_dir + 'Person.csv')
df_Blockgroup_UrbanVillage = pd.read_csv(data_dir + 'Blockgroup_UrbanVillage.csv')

#print (df_Household.head())
print (df_Blockgroup_UrbanVillage.head())


/Users/xx/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,18,21,24,25,26,28,29,31,33,90,112) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


     BLOCKGROUP URBAN_VILLAGE_NAME URBAN_VILLAGE_TYPE  \
0  530330001001   Outside Villages   Outside Villages   
1  530330001002   Outside Villages   Outside Villages   
2  530330001003   Outside Villages   Outside Villages   
3  530330001004   Outside Villages   Outside Villages   
4  530330001005   Outside Villages   Outside Villages   

                              NEIGHBORHOODS_INCLUDED  \
0  Cedar Park, Lake City, Meadowbrook, Matthews B...   
1  Cedar Park, Lake City, Meadowbrook, Matthews B...   
2  Cedar Park, Lake City, Meadowbrook, Matthews B...   
3  Cedar Park, Lake City, Meadowbrook, Matthews B...   
4  Cedar Park, Lake City, Meadowbrook, Matthews B...   

   NEIGHBORHOOD_DISTRICT_NUMBER NEIGHBORHOOD_DISTRICT_NAME  
0                             8                      North  
1                             8                      North  
2                             8                      North  
3                             8                      North  
4              

## Process Persons (Race demographics)

In [3]:
# load race information
df_persons = df_Person[['hhid','race_afam','race_aiak','race_asian','race_hapi','race_hisp','race_white','race_other']]
df_persons.dropna(axis = 0, inplace=True)

#race_afam	race_aiak	race_asian	race_hapi	race_hisp	race_white	race_other

df_race = df_persons.groupby(['hhid'], as_index=False).agg({'race_afam':sum,'race_aiak':sum,
                                                        'race_asian':sum,'race_hapi':sum,
                                                            'race_white':sum,'race_other':sum,
                                                           'race_hisp':sum})

df_race['race_total'] = df_race['race_afam'] + df_race['race_asian'] + df_race['race_hisp'] + df_race['race_white'] + df_race['race_other']
df_race['pct_white'] = df_race['race_white'] / df_race['race_total']
print (df_race)


          hhid  race_afam  race_aiak  race_asian  race_hapi  race_white  \
0     17100005        0.0        0.0         0.0        0.0         2.0   
1     17100024        0.0        0.0         0.0        0.0         2.0   
2     17100052        0.0        0.0         0.0        0.0         1.0   
3     17100059        0.0        0.0         0.0        0.0         0.0   
4     17100060        0.0        0.0         0.0        0.0         1.0   
5     17100102        0.0        0.0         1.0        0.0         1.0   
6     17100108        0.0        0.0         0.0        0.0         0.0   
7     17100111        0.0        0.0         1.0        0.0         2.0   
8     17100137        0.0        0.0         0.0        0.0         1.0   
9     17100149        0.0        0.0         2.0        0.0         0.0   
10    17100179        0.0        0.0         0.0        0.0         2.0   
11    17100203        0.0        0.0         2.0        0.0         0.0   
12    17100208        0.0

/Users/xx/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Process Households

In [4]:
# load household data
df_households = df_Household[['bg_household','hhid','final_lat','final_lng','hhsize','vehicle_count','numchildren',
                 'hhincome_broad','car_share','rent_own','res_dur','offpark','hh_wt_final']]

df_households['bg_household'] = df_households['bg_household'].astype(float).astype(int).astype(str)
# merge with seattle block group data
df_seattle = df_Blockgroup_UrbanVillage[['BLOCKGROUP','URBAN_VILLAGE_NAME','URBAN_VILLAGE_TYPE']]
df_seattle['bg_household'] = df_seattle['BLOCKGROUP'].astype(str)

df_households = pd.merge(left=df_households, right=df_seattle, how='left', on='bg_household')
df_households.drop(['BLOCKGROUP'], axis = 1, inplace=True)
df_households['URBAN_VILLAGE_NAME'] = df_households['URBAN_VILLAGE_NAME'].fillna("Outside Seattle")
df_households['URBAN_VILLAGE_TYPE'] = df_households['URBAN_VILLAGE_TYPE'].fillna("Outside Seattle")

df_households = df_households.rename(columns={'final_lat':'hh_lat', 'final_lng':'hh_lng'})
df_households = df_households.rename(columns={'URBAN_VILLAGE_NAME':'hh_uv', 'URBAN_VILLAGE_TYPE':'hh_uv_type'})

# Assign income variables
df_households['income'] = np.where(df_households['hhincome_broad']==1, "Under $25,000", "")
df_households['income'] = np.where(df_households['hhincome_broad']==2, "$25,000-$49,999", df_households['income'])
df_households['income'] = np.where(df_households['hhincome_broad']==3, "$50,000-$74,999", df_households['income'])
df_households['income'] = np.where(df_households['hhincome_broad']==4, "$75,000-$99,999", df_households['income'])
df_households['income'] = np.where(df_households['hhincome_broad']==5, "$100,000 or more", df_households['income'])
df_households['income'] = np.where(df_households['hhincome_broad']==6, "Prefer not to answer", df_households['income'])

# Assign home ownership
df_households['homeownership'] = np.where(df_households['rent_own']==1, "Own", "Other")
df_households['homeownership'] = np.where(df_households['rent_own']==2, "Rent", df_households['homeownership'])

# Assign residency tenure
df_households['tenure'] = np.where(df_households['res_dur']==1, "Less than a year", "")
df_households['tenure'] = np.where(df_households['res_dur']==2, "Between 1 and 2 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==3, "Between 2 and 3 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==4, "Between 3 and 5 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==5, "Between 5 and 10 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==6, "Between 10 and 20 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==7, "More than 20 years", df_households['tenure'])

df_households = pd.merge(left=df_households, right=df_race, how='left', on='hhid')

print (df_households.head())
print (df_households)

/Users/xx/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/xx/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


   bg_household      hhid     hh_lat      hh_lng  hhsize  vehicle_count  \
0  530330323234  17100005  47.695195 -122.099114       2              2   
1  530330076002  17100024  47.619659 -122.304511       3              1   
2  530330075005  17100052  47.620380 -122.318625       1              0   
3  530330084001  17100059  47.614840 -122.324427       1              0   
4  530530720002  17100060  47.120867 -122.527610       1              1   

   numchildren  hhincome_broad  car_share  rent_own    ...      \
0            0               4          2         1    ...       
1            1               4          1         1    ...       
2            0               1          2         2    ...       
3            0               5          2         2    ...       
4            0               1          2         2    ...       

                    tenure  race_afam  race_aiak race_asian race_hapi  \
0       More than 20 years        0.0        0.0        0.0       0.0   
1    B

## Process Trips

In [5]:
df_trips = df_Trip[['tripid','hhid','origin_lat','origin_lng','dest_lat','dest_lng',
                    'bg_origin','bg_dest','personid','google_duration','trip_path_distance',
                    'depart_time_timestamp','arrival_time_timestamp','daynum','o_purp','d_purp',
                    'mode_1','travelers_total','traveldate','trip_wt_final']]

df_trips['bg_dest'] = df_trips['bg_dest'].astype(str)
df_trips['bg_origin'] = df_trips['bg_origin'].astype(str)


# merge with seattle block group data
df_seattle = df_Blockgroup_UrbanVillage[['BLOCKGROUP','URBAN_VILLAGE_NAME','URBAN_VILLAGE_TYPE']]
df_seattle['BLOCKGROUP'] = df_seattle['BLOCKGROUP'].astype(str)
                              
df_trips = pd.merge(left=df_trips, right=df_seattle, how='left', left_on='bg_origin', right_on='BLOCKGROUP')
df_trips = df_trips.rename(columns={'URBAN_VILLAGE_NAME':'uv_origin', 'URBAN_VILLAGE_TYPE':'uvType_origin'})
df_trips.drop(['BLOCKGROUP'], axis = 1, inplace=True)

df_trips['uv_origin'] = df_trips['uv_origin'].fillna("Outside Seattle")
df_trips['uvType_origin'] = df_trips['uvType_origin'].fillna("Outside Seattle")

df_trips = pd.merge(left=df_trips, right=df_seattle, how='left', left_on='bg_dest', right_on='BLOCKGROUP')
df_trips = df_trips.rename(columns={'URBAN_VILLAGE_NAME':'uv_dest', 'URBAN_VILLAGE_TYPE':'uvType_dest'})
df_trips.drop(['BLOCKGROUP'], axis = 1, inplace=True)

df_trips['uv_dest'] = df_trips['uv_dest'].fillna("Outside Seattle")
df_trips['uvType_dest'] = df_trips['uvType_dest'].fillna("Outside Seattle")

# Drop missing variables, clean up column
df_trips['mode_1'] = df_trips['mode_1'].fillna(0)
df_trips['mode_1'] = df_trips['mode_1'].astype(str).replace(' ', '0')
df_trips['mode_1'] = df_trips['mode_1'].astype(str).astype(int)
df_trips['travelers_total'] = df_trips['travelers_total'].astype(str).replace(' ', '0')
df_trips['travelers_total'] = df_trips['travelers_total'].astype(str).astype(int)

# drop rows where the duration or distance is null or an empty space
df_trips = df_trips[df_trips['google_duration'].notnull()]
df_trips = df_trips[df_trips['trip_path_distance'].notnull()]
df_trips = df_trips[df_trips['google_duration'] != " "]
df_trips = df_trips[df_trips['trip_path_distance'] != " "]
df_trips['google_duration'] = df_trips['google_duration'].astype(float)
df_trips['trip_path_distance'] = df_trips['trip_path_distance'].astype(float)
df_trips = df_trips[df_trips['trip_path_distance'].notnull()]

# Create OD Pairs for urban villages and block groups
df_trips['uv_od_pair'] = df_trips['uv_origin'] + " to " + df_trips['uv_dest']
df_trips['bg_od_pair'] = df_trips['bg_origin'].astype(str) + " to " + df_trips['bg_dest'].astype(str)

# Assign mode variables
df_trips['mode'] = np.where(df_trips['mode_1']==1, "Walk", "Other")
df_trips['mode'] = np.where(df_trips['mode_1']==2, "Bike", df_trips['mode'])

df_trips['mode'] = np.where((df_trips['mode_1']>=3) & (df_trips['mode_1']<=17)  & (df_trips['travelers_total']==1), "Drive Alone", df_trips['mode'])
df_trips['mode'] = np.where(((df_trips['mode_1']==21) | (df_trips['mode_1']==22) |
                                 (df_trips['mode_1']==33) | (df_trips['mode_1']==34) | (df_trips['mode_1']==18))
                                 & (df_trips['travelers_total']==1), "Drive Alone", df_trips['mode'])

df_trips['mode'] = np.where((df_trips['mode_1']>=3) & (df_trips['mode_1']<=17)  & (df_trips['travelers_total']!=1), "Drive w Others", df_trips['mode'])
df_trips['mode'] = np.where(((df_trips['mode_1']==21) | (df_trips['mode_1']==22) |
                                 (df_trips['mode_1']==33) | (df_trips['mode_1']==34) | (df_trips['mode_1']==18))
                                 & (df_trips['travelers_total']>1), "Drive w Others", df_trips['mode'])

df_trips['mode'] = np.where((df_trips['mode_1']==23) | (df_trips['mode_1']==41) | (df_trips['travelers_total']==42), "Transit", df_trips['mode'])
df_trips['mode'] = np.where((df_trips['mode_1']==32) | (df_trips['travelers_total']==52), "Transit", df_trips['mode'])

df_trips['drive_alone'] = np.where(df_trips['mode']=="Drive Alone", 1, 0)


# Assign purpose variables
df_trips['purpose'] = np.where(df_trips['d_purp']==1, "Go Home", "Other")
df_trips['purpose'] = np.where(df_trips['d_purp']==6, "School", df_trips['purpose'])
df_trips['purpose'] = np.where((df_trips['d_purp']==10) | (df_trips['d_purp']==11), "Work", df_trips['purpose'])

# Assign time period variables
df_trips['depart_time_timestamp'] = pd.to_datetime(df_trips['depart_time_timestamp'], errors='coerce')
df_trips['depart_day'] = df_trips['depart_time_timestamp'].dt.dayofweek
df_trips['depart_time'] = df_trips['depart_time_timestamp'].dt.hour

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']>=7) & (df_trips['depart_time']<=9), "Weekday AM", "")

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']>=10) & (df_trips['depart_time']<=15), "Weekday Mid", df_trips['depart_period'])

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']>=16) & (df_trips['depart_time']<=19), "Weekday PM", df_trips['depart_period'])

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']<7), "Other", df_trips['depart_period'])

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']>19), "Late Night", df_trips['depart_period'])

df_trips['depart_period'] = np.where((df_trips['depart_day']>=5) & (df_trips['depart_day']<=6), "Late Night", df_trips['depart_period'])

#dts = dfBad[cols].apply(lambda x: pd.to_datetime(x, errors='coerce', format='%m/%d/%Y'))
#pd.to_datetime(df_trips['depart_time_timestamp'])

#df_trips['depart_time_timestamp'] = datetime.strptime(df_trips['depart_time_timestamp'], '%b %d %Y %I:%M%p')
#to_datetime

#print (df_trips.dtypes)
print (df_trips.head())

/Users/xx/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/xx/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/xx/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

          tripid      hhid  origin_lat  origin_lng   dest_lat   dest_lng  \
2   1.710000e+12  17119958   47.707597 -122.118828  999.00000  999.00000   
3   1.710000e+12  17119958   47.707597 -122.118828  999.00000  999.00000   
4   1.710000e+12  17119958   47.707597 -122.118828  999.00000  999.00000   
9   1.710000e+12  17108513   34.857550  132.033550   31.68794  124.34331   
10  1.710000e+12  17108513   31.150120  121.798040   31.31629  120.66029   

       bg_origin bg_dest    personid  google_duration      ...        \
2   530330323213    none  1711995801              4.0      ...         
3   530330323213    none  1711995803              4.0      ...         
4   530330323213    none  1711995804              4.0      ...         
9           none    none  1710851301             54.0      ...         
10          none    none  1710851301            118.0      ...         

            uv_dest      uvType_dest                          uv_od_pair  \
2   Outside Seattle  Outside Seatt

## Origin and Destination Summary

In [6]:
#filter for only trips starting and ending in Seattle
df_trips_Seattle = df_trips[(df_trips['uv_origin'] != 'Outside Seattle') | (df_trips['uv_dest'] != 'Outside Seattle')]

#print (df_trips_Seattle)

#aggregate by urban village origin
df_UV_Origins = df_trips_Seattle.groupby(['uv_origin'], as_index=False).agg({'google_duration':['mean','std','skew'],'trip_path_distance':['mean'],
                                                                            'origin_lat':['mean','std','skew'],'origin_lng':['mean']})

print (df_UV_Origins.head())

#aggregate by urban village destination
df_UV_Destinations = df_trips_Seattle.groupby(['uv_dest'], as_index=False).agg({'google_duration':['mean'],'trip_path_distance':['mean'],
                                                                                          'dest_lat':['mean'],'dest_lng':['mean']})
#print (df_UV_Destinations.head())

#aggregate by urban village OD Pair
df_UV_ODPair = df_trips_Seattle.groupby(['uv_od_pair'], as_index=False).agg({'google_duration':['mean'],'trip_path_distance':['mean'],
                                                                                          'origin_lat':['mean'],'origin_lat':['mean'],
                                                                                          'dest_lat':['mean'],'dest_lng':['mean']})
#print (df_UV_ODPair.head())

##aggregate by blockgroup origin
df_BG_Origins = df_trips_Seattle.groupby(['bg_origin'], as_index=False).agg({'google_duration':['mean'],'trip_path_distance':['mean'],
                                                                            'origin_lat':['mean'],'origin_lng':['mean']})
#print (df_BG_Origins)

#aggregate by blockgroup destination
df_BG_Destinations = df_trips_Seattle.groupby(['bg_dest'], as_index=False).agg({'google_duration':['mean'],'trip_path_distance':['mean'],
                                                                                          'dest_lat':['mean'],'dest_lng':['mean']})
#print (df_BG_Destinations.head())

#aggregate by urban village OD Pair
df_BG_ODPair = df_trips_Seattle.groupby(['bg_od_pair'], as_index=False).agg({'google_duration':['mean'],'trip_path_distance':['mean'],
                                                                            'origin_lat':['mean'],'origin_lng':['mean'],
                                                                             'dest_lat':['mean'],'dest_lng':['mean']})
df_BG_ODPair.columns = df_BG_ODPair.columns.droplevel(level=1)

df_BG_ODPair.to_csv(data_dir + 'df_BG_ODPair.csv', mode='w', header=True, index=False)
df_BG_Origins.to_csv(data_dir + 'BG_Origins.csv', mode='w', header=True, index=False)



               uv_origin google_duration                       \
                                    mean        std      skew   
0            12th Avenue       16.684028  15.493985  3.433822   
1   23rd & Union-Jackson       15.564369  14.651732  3.482621   
2                Admiral       13.990654  11.758173  1.232974   
3  Aurora-Licton Springs       16.343915  13.910217  2.610908   
4                Ballard       14.523540  14.356801  4.427405   

  trip_path_distance origin_lat                      origin_lng  
                mean       mean       std      skew        mean  
0           4.193542  47.608315  0.002951 -0.462030 -122.317345  
1           3.263156  47.603145  0.005409  1.487280 -122.306682  
2           3.490138  47.582350  0.000786  2.643821 -122.386420  
3           4.471715  47.696854  0.005029  0.480625 -122.345977  
4           3.437674  47.670593  0.003032  0.103379 -122.382603  


## Merged, Normalized Trip Dataset

In [34]:
df_trip_household= pd.merge(left=df_trips, right=df_households, how='left', left_on='hhid', right_on='hhid')

print (df_trip_household.head())

df_trip_household.to_csv(data_dir + 'Trip_Household_Merged.csv', mode='w', header=True, index=False)
df_UV_Origins.to_csv(data_dir + 'UV_Origins.csv', mode='w', header=True, index=False)

         tripid      hhid  origin_lat  origin_lng   dest_lat   dest_lng  \
0  1.710000e+12  17119958   47.707597 -122.118828  999.00000  999.00000   
1  1.710000e+12  17119958   47.707597 -122.118828  999.00000  999.00000   
2  1.710000e+12  17119958   47.707597 -122.118828  999.00000  999.00000   
3  1.710000e+12  17108513   34.857550  132.033550   31.68794  124.34331   
4  1.710000e+12  17108513   31.150120  121.798040   31.31629  120.66029   

      bg_origin bg_dest    personid  google_duration    ...      \
0  530330323213    none  1711995801              4.0    ...       
1  530330323213    none  1711995803              4.0    ...       
2  530330323213    none  1711995804              4.0    ...       
3          none    none  1710851301             54.0    ...       
4          none    none  1710851301            118.0    ...       

                  tenure race_afam race_aiak  race_asian race_hapi race_white  \
0  Between 3 and 5 years       0.0       1.0         2.0       0.